In [1]:
import os
import pandas as pd
import openai
import mistralai
import anthropic

c:\Users\Admin\anaconda3\envs\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

M_api_key = os.environ["MISTRAL_API_KEY"]
model = "mistral-large-latest"

client = MistralClient(api_key=M_api_key)

messages = [
    ChatMessage(role="user", content="What is the best French cheese?")
]

chat_response = client.chat(
    model=model,
    messages=messages,
)

print(chat_response.choices[0].message.content)

KeyError: 'MISTRAL_API_KEY'